In [1]:
from urllib.request import Request, urlopen
from bs4 import BeautifulSoup
import re
import pandas as pd
import csv
import traceback
import os
import urllib.parse
import sys
import time

In [2]:
# 保管先フォルダの名称
OUTPUT_DIR = 'livedoor_news'
# ニュース検索（ライブドアニュース）のURL
URL = 'https://news.livedoor.com/search/article/?p={1:}&word={0:}'
# ニュースID抽出用の正規表現（名前付きグループ）
REG_URL = r'(?P<L1>(https?://[^/]+/))(?P<L2>([^/]+))/(?P<L3>([^/]+))/(?P<L4>([^/]+))/'
# UserAgent
USER_AGENT = 'Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/94.0.4606.71 Safari/537.36'

In [3]:
def create_folder():
    if not os.path.exists(OUTPUT_DIR):
        # フォルダが存在しなければ、フォルダを作成
        os.makedirs(OUTPUT_DIR)

In [4]:
def get_links_paging(query=None, page=None):
    print(URL.format(urllib.parse.quote_plus(query, encoding='euc-jp'), page))
    try:
        # HTTPリクエストヘッダーにUser Agentを設定
        req = Request(URL.format(urllib.parse.quote_plus(query, encoding='euc-jp'), page), data=None, headers={'User-Agent': USER_AGENT})
        with urlopen(req) as res:
            # HTMLドキュメントからBeautifulSoupを初期化
            soup = BeautifulSoup(res.read().decode('euc_jp', 'ignore'), 'html.parser')
            # 存在しないページに辿り着いたら、中断（すべての結果を取得済み）
            if soup.find('div', id='content').find('p', class_='empty'):
                #print(soup.find('div', id='content').find('p', class_='empty').text)
                return True
            # 検索結果のニュース部分を検索し、全てを取得
            soupNewsQuery = soup.find('ul', class_='articleList').find_all('li')

            articles = []
            for idx, soupNews in enumerate(soupNewsQuery):
                # 詳細ページURLをHTMLタグの属性から抽出
                url = soupNews.a.get('href')
                # NewsBodyを検索し取得
                soupNewsBody = soupNews.find('div', class_='articleListBody')
                # NewsBodyから各種データを抽出
                article = {
                    'url': url,
                    # ニュースIDを詳細ページURLから抽出
                    'id': re.search(REG_URL, url).groupdict()['L4'],
                    # タイトル／サマリ／提供元／公開日時をHTMLタグの本文（公開日時だけ属性）から抽出
                    'title': soupNewsBody.find('h3', class_='articleListTtl').text,
                    'summary': soupNewsBody.find('p', class_='articleListSummary').text,
                    'vender': soupNewsBody.find('p', class_='articleListVender').text,
                    'datetime': soupNewsBody.find('time', class_='articleListDate').get('datetime')
                }
                articles.append(article)
                #print('%2d: %s' %(idx + 1, soupNewsBody.find('h3', class_='articleListTtl').text))

            df = pd.DataFrame(articles)
            # DataFrameからCSVファイルを生成
            # encoding='sjis'だとJupyterLab（CSVTable）上で表示不可なことに注意
            df.to_csv('%s/livedoor_news_query_%s.csv' %(OUTPUT_DIR, query), encoding='utf-8', index=False, quotechar='"', quoting=csv.QUOTE_ALL, mode='a', header=False)
            return False
    except Exception as e:
        # エラー概要
        print('Exception: ', e)
        print('=====')
        # エラー詳細（スタックトレース）
        print(traceback.format_exc().rstrip())
        print('=====')

In [5]:
def get_links(query=None):
    try:
        for page in range(1, sys.maxsize):
            isFinish = get_links_paging(query, page)
            if isFinish:
                return
            time.sleep(5)
    except Exception as e:
        # エラー概要
        print('Exception: ', e)
        print('=====')
        # エラー詳細（スタックトレース）
        print(traceback.format_exc().rstrip())
        print('=====')   

In [6]:
create_folder()
get_links('緊急事態宣言 解除')

https://news.livedoor.com/search/article/?p=1&word=%B6%DB%B5%DE%BB%F6%C2%D6%C0%EB%B8%C0+%B2%F2%BD%FC
https://news.livedoor.com/search/article/?p=2&word=%B6%DB%B5%DE%BB%F6%C2%D6%C0%EB%B8%C0+%B2%F2%BD%FC
https://news.livedoor.com/search/article/?p=3&word=%B6%DB%B5%DE%BB%F6%C2%D6%C0%EB%B8%C0+%B2%F2%BD%FC
https://news.livedoor.com/search/article/?p=4&word=%B6%DB%B5%DE%BB%F6%C2%D6%C0%EB%B8%C0+%B2%F2%BD%FC
https://news.livedoor.com/search/article/?p=5&word=%B6%DB%B5%DE%BB%F6%C2%D6%C0%EB%B8%C0+%B2%F2%BD%FC
https://news.livedoor.com/search/article/?p=6&word=%B6%DB%B5%DE%BB%F6%C2%D6%C0%EB%B8%C0+%B2%F2%BD%FC
https://news.livedoor.com/search/article/?p=7&word=%B6%DB%B5%DE%BB%F6%C2%D6%C0%EB%B8%C0+%B2%F2%BD%FC
https://news.livedoor.com/search/article/?p=8&word=%B6%DB%B5%DE%BB%F6%C2%D6%C0%EB%B8%C0+%B2%F2%BD%FC
https://news.livedoor.com/search/article/?p=9&word=%B6%DB%B5%DE%BB%F6%C2%D6%C0%EB%B8%C0+%B2%F2%BD%FC
https://news.livedoor.com/search/article/?p=10&word=%B6%DB%B5%DE%BB%F6%C2%D6%C0%EB%B8%C0+%B

In [7]:
# CSVファイルからDataFrameを生成
df = pd.read_csv('livedoor_news/livedoor_news_query_緊急事態宣言 解除.csv', encoding='utf-8', names=['url', 'id', 'title', 'summary', 'vender', 'datetime'])
df.head()

,url,id,title,summary,vender,datetime
0,https://news.livedoor.com/article/detail/20998...,20998048,酒類提供、１１日再開＝東京ディズニー,オリエンタルランドは８日、運営する東京ディズニーランドと東京ディズニーシー（いずれも千葉県浦...,時事通信社,2021-10-08 21:32:58
1,https://news.livedoor.com/article/detail/20997...,20997870,第２０５国会 岸田文雄首相所信表明演説全文,一、はじめに第２０５回国会の開会にあたり、新型コロナウイルスにより亡くなられた方々、そして、...,産経新聞,2021-10-08 20:57:28
2,https://news.livedoor.com/article/detail/20997...,20997874,経済“立て直し”へ お得なキャンペーンも,東京では８日、新たに１３８人の感染が確認され、７日連続で２００人を下回りました。減少傾向が続...,日テレNEWS24,2021-10-08 20:54:33
3,https://news.livedoor.com/article/detail/20997...,20997939,【独自】福岡県が15日に時短要請全面解除 約2カ月半ぶり、11日にも正式決定,福岡県は8日、新型コロナウイルス対策で実施している飲食店への営業時間短縮要請を14日の期限で...,西日本新聞,2021-10-08 20:39:00
4,https://news.livedoor.com/article/detail/20997...,20997159,櫻井＆相葉 あらゆる分野での功績がある嵐だからこそのW結婚発表,9月28日、嵐の櫻井翔と相葉雅紀がそろって結婚を発表した。これまで、嵐の陰の努力を見続けてき...,NEWSポストセブン,2021-10-08 19:05:00


In [8]:
get_links('コロナ')

https://news.livedoor.com/search/article/?p=1&word=%A5%B3%A5%ED%A5%CA
https://news.livedoor.com/search/article/?p=2&word=%A5%B3%A5%ED%A5%CA
https://news.livedoor.com/search/article/?p=3&word=%A5%B3%A5%ED%A5%CA
https://news.livedoor.com/search/article/?p=4&word=%A5%B3%A5%ED%A5%CA
https://news.livedoor.com/search/article/?p=5&word=%A5%B3%A5%ED%A5%CA
https://news.livedoor.com/search/article/?p=6&word=%A5%B3%A5%ED%A5%CA
https://news.livedoor.com/search/article/?p=7&word=%A5%B3%A5%ED%A5%CA
https://news.livedoor.com/search/article/?p=8&word=%A5%B3%A5%ED%A5%CA
https://news.livedoor.com/search/article/?p=9&word=%A5%B3%A5%ED%A5%CA
https://news.livedoor.com/search/article/?p=10&word=%A5%B3%A5%ED%A5%CA
https://news.livedoor.com/search/article/?p=11&word=%A5%B3%A5%ED%A5%CA
https://news.livedoor.com/search/article/?p=12&word=%A5%B3%A5%ED%A5%CA
https://news.livedoor.com/search/article/?p=13&word=%A5%B3%A5%ED%A5%CA
https://news.livedoor.com/search/article/?p=14&word=%A5%B3%A5%ED%A5%CA
https://news.li